In [1]:
import numpy as np
import pandas as pd
import logging
import sys
from typing import Union

from src.staff.hermes import Hermes

# configure logger
LOG_LEVEL = "DEBUG"
LOGGER = logging.getLogger('main')
LOGGER.setLevel(LOG_LEVEL)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(funcName)s [%(levelname)s]: %(message)s'
)
ch.setFormatter(formatter)

LOGGER.addHandler(ch)

In [2]:
def get_data(symbol: str, interval: str, start: Union[str, int], 
                end: Union[str, int]):
    
    exchange = 'kucoin' if '-' in symbol else 'binance'
    hermes = Hermes(exchange=exchange, mode='live')
    
    symbol = symbol.upper()

    res = hermes.get_ohlcv(
        symbols=symbol, interval=interval, start=start, end=end
    )
    
    if isinstance(res, dict) and res.get('success'):
        return res.get('message')
    else:
        LOGGER.error('unable to get OHLCV data')
        LOGGER.error(res)
        sys.exit()


In [7]:
symbol = 'btcusdt'
interval = '15m'
start = -200_000
end = 'now UTC'

dir = f'./ohlcv_data/'
filename = f'{dir}{symbol}_{interval}.csv'
key = f'{symbol}_{interval}'

data: pd.DataFrame = get_data(symbol, interval, start, end)
data 

2023-08-01 13:19:28,036 - main.hermes - __init__ [DEBUG]: HERMES initializing in live mode:
2023-08-01 13:19:28,045 - main.hermes - _initialize_symbol_cache [DEBUG]: initializing symbol cache ...


2023-08-01 13:19:28,270 - main.hermes - _initialize_symbol_cache [DEBUG]: loaded symbols from database for binance
2023-08-01 13:19:28,281 - main.hermes - get_ohlcv [INFO]: fetching data for ['BTCUSDT'] on BINANCE for period: 2017-11-17 03:19:28 - 2023-08-01 11:19:28
2023-08-01 13:19:28,281 - main.hermes - _get_ohlcv_for_one_symbol [DEBUG]: trying to retrieve data from database (1510888768000, 1690888768000)
2023-08-01 13:19:32,275 - main.hermes - _get_ohlcv_for_one_symbol [DEBUG]: 1690888499999 < 1690888768000: True
2023-08-01 13:19:32,364 - main.hermes - _update_ohlcv_table [DEBUG]: table status: {'name': 'binance_BTCUSDT_ohlcv_15m', 'table exists': True, 'number of entries': 195794, 'earliest open': 1502942400000, 'latest open': 1690887600000, 'symbol': 'BTCUSDT', 'interval': '15m'}
2023-08-01 13:19:32,364 - main.hermes - _update_ohlcv_table [DEBUG]: no update necessary for BTCUSDT - 15m (latest open time: 2023-08-01 11:00:00)
2023-08-01 13:19:32,364 - main.hermes - _get_ohlcv_for_o

,open time,human open time,open,high,low,close,volume,close time,quote asset volume
0,1510889400000,2017-11-17 04:30:00,7930.66,7970.00,7930.66,7941.45,37.329247,1510890299999,2.965987e+05
1,1510890300000,2017-11-17 04:45:00,7941.48,7962.98,7867.20,7890.00,27.910216,1510891199999,2.212887e+05
2,1510891200000,2017-11-17 05:00:00,7894.98,7930.80,7852.09,7868.15,61.798440,1510892099999,4.871294e+05
3,1510892100000,2017-11-17 05:15:00,7863.95,7885.78,7853.79,7883.78,60.378012,1510892999999,4.750280e+05
4,1510893000000,2017-11-17 05:30:00,7883.80,7901.45,7820.15,7820.15,28.996001,1510893899999,2.279944e+05
...,...,...,...,...,...,...,...,...,...
186987,1690877700000,2023-08-01 08:15:00,28921.50,28966.52,28907.04,28958.41,246.433310,1690878599999,7.129584e+06
186988,1690884900000,2023-08-01 10:15:00,28913.75,28913.75,28900.00,28907.98,468.987730,1690885799999,1.355798e+07
186989,1690885800000,2023-08-01 10:30:00,28907.97,28927.68,28907.97,28927.67,140.073980,1690886699999,4.050846e+06
186990,1690886700000,2023-08-01 10:45:00,28927.67,28946.00,28925.92,28928.01,237.442810,1690887599999,6.870253e+06


In [8]:
print(f'writing data to {filename} ...')

data.to_csv(filename, index=True)

writing data to ./ohlcv_data/btcusdt_15m.csv ...
